# Noca Nocca

In [22]:
from games.nocca_nocca.nocca_nocca import NoccaNocca
from agents.agent_random import RandomAgent
from agents.minimax import MiniMax   
from agents.mcts_t import MonteCarloTreeSearch

# Mini Max

In [23]:
game = NoccaNocca(max_steps=150, initial_player=0, seed=1)

In [24]:
agents = {
    game.agents[0]: MiniMax(game=game, agent=game.agents[0], depth=1),
    game.agents[1]: MiniMax(game=game, agent=game.agents[1], depth=1),
}

In [25]:
game.reset()
print(f"Initial Agent: {game.agent_selection}")
while not game.game_over():
    game.render()
    action = agents[game.agent_selection].action()
    print(f"Turn {game.steps} -- Agent {game.agent_selection} plays action {action}")
    game.step(action=action)
game.render()
if game.truncated():
    print("Game was truncated")
for agent in agents:
    print(f"Reward agent {agent}: {game.reward(agent)}")
print(f"The winner is: {game.check_for_winner()}")

Initial Agent: Black
0: ___ ___ ___ ___ ___ 
1: 0__ 0__ 0__ 0__ 0__ 
2: ___ ___ ___ ___ ___ 
3: ___ ___ ___ ___ ___ 
4: ___ ___ ___ ___ ___ 
5: ___ ___ ___ ___ ___ 
6: 1__ 1__ 1__ 1__ 1__ 
7: ___ ___ ___ ___ ___ 
Turn 0 -- Agent Black plays action 62
0: ___ ___ ___ ___ ___ 
1: 0__ 0__ ___ 0__ 0__ 
2: ___ ___ ___ 0__ ___ 
3: ___ ___ ___ ___ ___ 
4: ___ ___ ___ ___ ___ 
5: ___ ___ ___ ___ ___ 
6: 1__ 1__ 1__ 1__ 1__ 
7: ___ ___ ___ ___ ___ 
Turn 1 -- Agent White plays action 253
0: ___ ___ ___ ___ ___ 
1: 0__ 0__ ___ 0__ 0__ 
2: ___ ___ ___ 0__ ___ 
3: ___ ___ ___ ___ ___ 
4: ___ ___ ___ ___ ___ 
5: 1__ ___ ___ ___ ___ 
6: 1__ ___ 1__ 1__ 1__ 
7: ___ ___ ___ ___ ___ 
Turn 2 -- Agent Black plays action 68
0: ___ ___ ___ ___ ___ 
1: 0__ 0__ ___ ___ 0__ 
2: ___ ___ 0__ 0__ ___ 
3: ___ ___ ___ ___ ___ 
4: ___ ___ ___ ___ ___ 
5: 1__ ___ ___ ___ ___ 
6: 1__ ___ 1__ 1__ 1__ 
7: ___ ___ ___ ___ ___ 
Turn 3 -- Agent White plays action 271
0: ___ ___ ___ ___ ___ 
1: 0__ 0__ ___ ___ 0__ 
2: ___ __

# Monte Carlo Tree Search

In [26]:
game = NoccaNocca(max_steps=150, initial_player=0, seed=1)

In [27]:
agents = {
    game.agents[0]: MonteCarloTreeSearch(game=game, agent=game.agents[0]),
    game.agents[1]: RandomAgent(game=game, agent=game.agents[1]),
}

In [ ]:
wins = {'Empate': 0.0, 'White': 0.0, 'Black': 0.0}
for i in range(10):
    game.reset()
    print(f"Initial Agent: {game.agent_selection}")
    while not game.game_over():
        game.render()
        action = agents[game.agent_selection].action()
        print(f"Turn {game.steps} -- Agent {game.agent_selection} plays action {action}")
        game.step(action=action)
    game.render()
    if game.truncated():
        print("Game was truncated")
    for agent in agents:
        print(f"Reward agent {agent}: {game.reward(agent)}")
    print(f"The winner is: {game.check_for_winner()}")
    winner = 'Empate' if game.check_for_winner() is None else game.check_for_winner()

    wins[winner] += 1

print("Results after 10 games:")
for agent, count in wins.items():
    print(f"Agent {agent} won {count} times.")

Initial Agent: Black
0: ___ ___ ___ ___ ___ 
1: 0__ 0__ 0__ 0__ 0__ 
2: ___ ___ ___ ___ ___ 
3: ___ ___ ___ ___ ___ 
4: ___ ___ ___ ___ ___ 
5: ___ ___ ___ ___ ___ 
6: 1__ 1__ 1__ 1__ 1__ 
7: ___ ___ ___ ___ ___ 
Turn 0 -- Agent Black plays action 50
0: ___ ___ ___ ___ ___ 
1: 00_ ___ 0__ 0__ 0__ 
2: ___ ___ ___ ___ ___ 
3: ___ ___ ___ ___ ___ 
4: ___ ___ ___ ___ ___ 
5: ___ ___ ___ ___ ___ 
6: 1__ 1__ 1__ 1__ 1__ 
7: ___ ___ ___ ___ ___ 
Turn 1 -- Agent White plays action 266
0: ___ ___ ___ ___ ___ 
1: 00_ ___ 0__ 0__ 0__ 
2: ___ ___ ___ ___ ___ 
3: ___ ___ ___ ___ ___ 
4: ___ ___ ___ ___ ___ 
5: ___ ___ ___ ___ ___ 
6: 1__ 1__ 11_ ___ 1__ 
7: ___ ___ ___ ___ ___ 
Turn 2 -- Agent Black plays action 46
0: ___ ___ ___ ___ ___ 
1: 0__ ___ 0__ 0__ 0__ 
2: ___ 0__ ___ ___ ___ 
3: ___ ___ ___ ___ ___ 
4: ___ ___ ___ ___ ___ 
5: ___ ___ ___ ___ ___ 
6: 1__ 1__ 11_ ___ 1__ 
7: ___ ___ ___ ___ ___ 
Turn 3 -- Agent White plays action 253
0: ___ ___ ___ ___ ___ 
1: 0__ ___ 0__ 0__ 0__ 
2: ___ 0_

KeyError: 'Black'